In [1]:
import main
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import time
import sklearn
import keras

Using TensorFlow backend.


## ANN Model

In [ ]:
import tensorflow as tf
import keras

class net():
    def __init__(self,network_sizes,activation=None,dropout=0,optimizer = 'adam',
                 loss = 'mean_squared_error',metrics = None, n_out = 1):
        
        self.model = keras.models.Sequential()
        
        #Pass elements from init
        self.__dict__.update(locals())
        
        #print(var_dict)
        
        for i in self.network_sizes:
            
            self.model.add(keras.layers.Dropout(self.dropout))
            self.model.add(keras.layers.BatchNormalization())
            self.model.add(keras.layers.Dense(i,activation = self.activation))
            
        self.model.add(keras.layers.Dense(n_out))
        
        if n_out > 1:
            self.model.add(keras.layers.Softmax())
        
        self.model.compile(loss = self.loss, optimizer=self.optimizer, metrics = self.metrics)
        

        
    def fit_model(self,x ,y , batch_size, epochs,validation_split, verbose = 1):
        self.x_train = x
        self.y_train = y
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.verbose = verbose
        
        self.model.fit(
          x = self.x_train,
          y = self.y_train,
          batch_size=self.batch_size,
          epochs=self.epochs,
          verbose=self.verbose,
          validation_split = self.validation_split
         )
        
    def predict(self,x):
        return self.model.predict(x)
    
    def eval(self,x,y):
        return  self.model.evaluate(x, y, verbose=self.verbose)
        

a = net([150,50],loss = 'categorical_crossentropy', 
        optimizer = 'adam',activation = 'relu',metrics = ['accuracy'],n_out = 10)    

a.fit_model(x_train.reshape(-1,(28*28)).astype('float32'),
            y_train,200,4,0.01)

a.eval(x_test.reshape(-1,(28*28)).astype('float32'),y_test)

In [78]:
df_train_new = pd.read_csv('/Users/matheus/Downloads/testfiles/train_new.csv')

In [187]:
def proc(frame,var,tipo):
    dados = frame[[var]]
    if tipo == 'str':
        le = LabelEncoder()
        if frame[var].dtype == np.object:
            dados = dados.fillna('bla')
        else:
            dados = dados.fillna(np.pi)
        dados = dados.values.reshape(-1)
        out = pd.get_dummies(le.fit_transform(dados)).values
    if tipo == 'number':
        sc = StandardScaler()
        out = sc.fit_transform(dados.values)
    if tipo == 'binary':
        out = dados.values
        
    return out

def proc_dic (frame,dic):
    out = []
    for i in dic.keys():
        n = proc(frame,i,dic[i])
        out.append(n)
    return np.concatenate(out,axis=1)

dic = {
    'NU_NOTA_CN':'number','NU_NOTA_CH':'number','NU_NOTA_LC':'number',
    ##Notas da redacao:
    'NU_NOTA_COMP1':'number','NU_NOTA_COMP2':'number','NU_NOTA_COMP3':'number',
    'NU_NOTA_COMP4':'number','NU_NOTA_COMP5':'number',
    'NU_NOTA_REDACAO':'number'
}

drop = [i for i in dic.keys() if dic[i]=='number' or dic[i]=='binary']
new = df_train_new.dropna(subset = drop)

x = proc_dic(new,dic)
y = new[['NU_NOTA_MT']].values
s = StandardScaler()
y =s.fit_transform(y)
print('x shape:',x.shape)
print('y shape:',y.shape)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=False)

x shape: (10097, 9)
y shape: (10097, 1)


In [191]:
a = net([20,10],loss = 'mean_squared_error', 
        optimizer = 'adam',activation = 'relu')   

a.fit_model(
    x_train,
    y_train,
    100,
    20,
    0.1
    )

Train on 8178 samples, validate on 909 samples
Epoch 1/20
8178/8178 [==============================] - 2s 277us/step - loss: 0.9579 - val_loss: 0.6780
Epoch 2/20
8178/8178 [==============================] - 0s 27us/step - loss: 0.6602 - val_loss: 0.6068
Epoch 3/20
8178/8178 [==============================] - 0s 26us/step - loss: 0.6203 - val_loss: 0.5784
Epoch 4/20
8178/8178 [==============================] - 0s 26us/step - loss: 0.6065 - val_loss: 0.5681
Epoch 5/20
8178/8178 [==============================] - 0s 25us/step - loss: 0.5943 - val_loss: 0.5595
Epoch 6/20
8178/8178 [==============================] - 0s 24us/step - loss: 0.5934 - val_loss: 0.5509
Epoch 7/20
8178/8178 [==============================] - 0s 25us/step - loss: 0.5814 - val_loss: 0.5478
Epoch 8/20
8178/8178 [==============================] - 0s 26us/step - loss: 0.5822 - val_loss: 0.5460
Epoch 9/20
8178/8178 [==============================] - 0s 32us/step - loss: 0.5800 - val_loss: 0.5416
Epoch 10/20
8178/8178 [==

# MNIST Using ANN Network

In [1]:
import main
import keras

#Load mnist dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#Convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#Rescale X
x_train /= 255
x_test /= 255
#Number of classes
num_classes = 10

#Convert to one hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Create model
a = main.ANN(
        network_sizes = [150,50],
        loss = 'categorical_crossentropy', 
        optimizer = 'adam',
        activation = 'relu',
        metrics = ['accuracy'],
        n_out = 10
        )    

#Train model
a.fit_model(
        x_train.reshape(-1,(28*28)).astype('float32'),
        y_train,
        batch_size = 400,
        epochs = 4,
        validation_split = 0.01
        )

#Evaluate model
a.eval(
        x_test.reshape(-1,(28*28)).astype('float32'),
        y_test
        )

Using TensorFlow backend.


Train on 59400 samples, validate on 600 samples
Epoch 1/4
59400/59400 [==============================] - 3s 44us/step - loss: 0.3860 - acc: 0.8858 - val_loss: 0.1564 - val_acc: 0.9617
Epoch 2/4
59400/59400 [==============================] - 2s 38us/step - loss: 0.1330 - acc: 0.9608 - val_loss: 0.1442 - val_acc: 0.9700
Epoch 3/4
59400/59400 [==============================] - 2s 29us/step - loss: 0.0837 - acc: 0.9758 - val_loss: 0.1423 - val_acc: 0.9717
Epoch 4/4
10000/10000 [==============================] - 0s 34us/step


[0.09290625496208668, 0.9718]

## CNN Model

In [12]:
import keras

class CNN():
    def __init__(
            self,
            network_sizes,
            filters,
            kernels,
            pool_size = (1,1),
            activation=None,
            dropout=0,
            optimizer = 'adam',
            loss = 'mean_squared_error',
            metrics = None,
            n_out = 1
           ):
        
        #Pass elements from init
        self.__dict__.update(locals())
        
        #Create model
        self.model = keras.models.Sequential()
        
        #Create Convolution
        for f,k in zip(self.filters,self.kernels):
            self.model.add(keras.layers.Conv2D(
                f, kernel_size= k ,activation = self.activation,))
     
        #Create MaxPooling
        self.model.add(keras.layers.MaxPooling2D(pool_size=self.pool_size))
        self.model.add(keras.layers.Dropout(self.dropout))
        self.model.add(keras.layers.Flatten())
        
        #Create Hidden Layers
        for i in self.network_sizes:
            self.model.add(keras.layers.Dropout(self.dropout))
            self.model.add(keras.layers.BatchNormalization())
            self.model.add(keras.layers.Dense(i,activation = self.activation))
        
        #Create output layer
        self.model.add(keras.layers.Dense(n_out))
        
        #If logits regression create softmax
        if n_out > 1:
            self.model.add(keras.layers.Softmax())
        
        #Compile model
        self.model.compile(loss = self.loss, optimizer=self.optimizer, metrics = self.metrics)
        
        
    def fit_model(
            self,
            x,
            y,
            batch_size,
            epochs,
            validation_split,
            verbose = 1
            ):
        
        self.x_train = x
        self.y_train = y
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.verbose = verbose
        
        #Fit model
        self.model.fit(
          x = self.x_train,
          y = self.y_train,
          batch_size=self.batch_size,
          epochs=self.epochs,
          verbose=self.verbose,
          validation_split = self.validation_split
         )
        
    def predict(self,x):
        return self.model.predict(x)
    
    def eval(self,x,y):
        return  self.model.evaluate(x, y, verbose=self.verbose)
        




# MNIST Using CNN Network

In [17]:
#Load mnist dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#Convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#Rescale X
x_train /= 255
x_test /= 255

#Number of classes
num_classes = 10

#Convert to one hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = CNN(
        network_sizes = [50,25],
        filters = [5,5],
        kernels = [(4,4),(2,2)],
        pool_size = (2,2),
        activation='relu',
        dropout=0,
        optimizer = 'adam',
        loss = 'categorical_crossentropy',
        metrics = ['accuracy'],
        n_out = 10
    )

model.fit_model(
        x = x_train.reshape(-1,28,28,1),
        y = y_train,
        batch_size = 200,
        epochs = 5,
        validation_split = 0.01,
        verbose = 1
)

Train on 59400 samples, validate on 600 samples
Epoch 1/5
59400/59400 [==============================] - 22s 373us/step - loss: 0.3643 - acc: 0.8984 - val_loss: 0.1220 - val_acc: 0.9667
Epoch 2/5
59400/59400 [==============================] - 19s 328us/step - loss: 0.0881 - acc: 0.9738 - val_loss: 0.0981 - val_acc: 0.9750
Epoch 3/5
59400/59400 [==============================] - 22s 364us/step - loss: 0.0578 - acc: 0.9829 - val_loss: 0.0885 - val_acc: 0.9783
Epoch 4/5
59400/59400 [==============================] - 23s 381us/step - loss: 0.0428 - acc: 0.9869 - val_loss: 0.0864 - val_acc: 0.9833
Epoch 5/5
59400/59400 [==============================] - 22s 367us/step - loss: 0.0324 - acc: 0.9902 - val_loss: 0.0934 - val_acc: 0.9783


In [40]:
import keras

class RNN():
    def __init__(
            self,
            network_sizes,
            lstm_cells,
            rnn_type = 'lstm',
            activation=None,
            dropout=0,
            optimizer = 'adam',
            loss = 'mean_squared_error',
            metrics = None,
            n_out = 1
           ):
        
        #Pass elements from init
        self.__dict__.update(locals())
        
        #Create model
        self.model = keras.models.Sequential()
        
        #Create LSTM Layer
        for i in self.lstm_cells:
            if self.rnn_type == 'lstm':
                self.model.add(keras.layers.LSTM(i,dropout = self.dropout, 
                    activation = self.activation))
            else:
                self.model.add(keras.layers.GRU(i,dropout = self.dropout, 
                    activation = self.activation))
        
        #Create Hidden Layers
        for i in self.network_sizes:
            self.model.add(keras.layers.Dropout(self.dropout))
            self.model.add(keras.layers.BatchNormalization())
            self.model.add(keras.layers.Dense(i,activation = self.activation))
        
        #Create output layer
        self.model.add(keras.layers.Dense(n_out))
        
        #If logits regression create softmax
        if n_out > 1:
            self.model.add(keras.layers.Softmax())
        
        #Compile model
        self.model.compile(loss = self.loss, optimizer=self.optimizer, metrics = self.metrics)
        
        
    def fit_model(
            self,
            x,
            y,
            batch_size,
            epochs,
            validation_split,
            verbose = 1
            ):
        
        self.x_train = x
        self.y_train = y
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.verbose = verbose
        
        #Fit model
        self.model.fit(
          x = self.x_train,
          y = self.y_train,
          batch_size=self.batch_size,
          epochs=self.epochs,
          verbose=self.verbose,
          validation_split = self.validation_split
         )
        
    def predict(self,x):
        return self.model.predict(x)
    
    def eval(self,x,y):
        return  self.model.evaluate(x, y, verbose=self.verbose)
        





# MNIST Using CNN Network

In [1]:
import main
import keras

#Load mnist dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#Convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#Rescale X
x_train /= 255
x_test /= 255

#Number of classes
num_classes = 10

#Convert to one hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = main.RNN(
    network_sizes = [30,15],
    lstm_cells = [28],
    activation='relu',
    rnn_type = 'GRU',
    dropout = 0.1,
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
    n_out = 10
)

model.fit_model(
    x = x_train,
    y = y_train,
    batch_size = 200,
    epochs = 5,
    validation_split = 0.01,
    verbose = 1
)



Using TensorFlow backend.


Train on 59400 samples, validate on 600 samples
Epoch 1/5
59400/59400 [==============================] - 9s 155us/step - loss: 1.6634 - acc: 0.4077 - val_loss: 1.0522 - val_acc: 0.6200
Epoch 2/5
59400/59400 [==============================] - 7s 117us/step - loss: 0.9813 - acc: 0.6523 - val_loss: 0.5316 - val_acc: 0.8033
Epoch 3/5
59400/59400 [==============================] - 7s 114us/step - loss: 0.6466 - acc: 0.7819 - val_loss: 0.3317 - val_acc: 0.8850
Epoch 4/5
59400/59400 [==============================] - 7s 122us/step - loss: 0.4747 - acc: 0.8476 - val_loss: 0.2335 - val_acc: 0.9167
Epoch 5/5
59400/59400 [==============================] - 7s 117us/step - loss: 0.3565 - acc: 0.8908 - val_loss: 0.1732 - val_acc: 0.9517
